# Alice CID categorization sheet export

The *gestão de saúde magenta* squad inputs dw weights for daly calculations manually in a google sheet. This notebook is to export the sheet, after downloading from the Google Drive and storing it in a local computer. Since the sheet is updated not very often, there is not much gain from automating the process in Airflow (*yet*).

Steps:

    - Download the google sheet DALY weights and store it in the 'redshift_import_export\external\' package folder;

    - run this notebook to upload to Redshift;
    
    - use the DW to make calculations, new tables, dashes...
***

## 0. Setup

In [2]:
#connection to redshift
from redshift_import_export.interfaces.data_interactor import DataInteractor
di = DataInteractor()
import requests


#standard packages
import warnings
warnings.filterwarnings('ignore')
import os
import collections
import pandas as pd
import numpy as np



#time packages
import datetime as dt

## 1. Import

In [23]:
df1 = pd.read_csv(os.path.abspath(os.path.join(os.getcwd(), os.pardir)) + '/data/external/alice_cid_categories.csv')
df2 = pd.read_csv(os.path.abspath(os.path.join(os.getcwd(), os.pardir)) + '/data/external/alice_acute_class.csv')

In [29]:
df1.columns
df1.rename(columns={'Disease.Code.Value': 'cid', 'Categoria 2.0': 'alice_category', 'capitulo': 'chapter', 'grupos': 'group', 'categorias': 'category', 'subcategorias': 'subcategory'}, inplace = True)
df1 = df1[['cid', 'alice_category', 'chapter', 'group', 'category', 'subcategory']]

In [30]:
df1

,cid,alice_category,chapter,group,category,subcategory
0,A01,NaN,Capítulo I - Algumas doenças infecciosas e par...,Doenças infecciosas intestinais,Febres tifóide e paratifóide,Febres tifóide e paratifóide
1,A02,Infecções gastrointestinais,Capítulo I - Algumas doenças infecciosas e par...,Doenças infecciosas intestinais,Outras infecções por Salmonella,Outras infecções por Salmonella
2,A022,NaN,Capítulo I - Algumas doenças infecciosas e par...,Doenças infecciosas intestinais,Outras infecções por Salmonella,Infecções localizadas por salmonela
3,A04,Infecções gastrointestinais,Capítulo I - Algumas doenças infecciosas e par...,Doenças infecciosas intestinais,Outras infecções intestinais bacterianas,Outras infecções intestinais bacterianas
4,A049,Infecções gastrointestinais,Capítulo I - Algumas doenças infecciosas e par...,Doenças infecciosas intestinais,Outras infecções intestinais bacterianas,Infecção intestinal bacteriana não especificada
...,...,...,...,...,...,...
2902,Z98,Fatores de risco e antecedentes pessoais,Capítulo XXI - Fatores que influenciam o estad...,Pessoas com riscos potenciais à saúde relacion...,Outros estados pós-cirúrgicos,Outros estados pós-cirúrgicos
2903,Z980,Fatores de risco e antecedentes pessoais,Capítulo XXI - Fatores que influenciam o estad...,Pessoas com riscos potenciais à saúde relacion...,Outros estados pós-cirúrgicos,Derivação intestinal ou anastomose intestinal
2904,Z981,Fatores de risco e antecedentes pessoais,Capítulo XXI - Fatores que influenciam o estad...,Pessoas com riscos potenciais à saúde relacion...,Outros estados pós-cirúrgicos,Artrodese
2905,Z982,Fatores de risco e antecedentes pessoais,Capítulo XXI - Fatores que influenciam o estad...,Pessoas com riscos potenciais à saúde relacion...,Outros estados pós-cirúrgicos,Presença de dispositivo de drenagem do líquor


## 2. Transform

In [25]:
df2.rename(columns = {'CID 10': 'cid','categoria alice': 'alice_category', 'agudo ou longitudinal?': 'alice_acute_class'}, inplace = True)

df2 = df2[df2['alice_category'].isna()!=True]
df2.drop_duplicates(subset= ['cid', 'alice_category'], inplace = True)

In [28]:
df3 = df1.merge(df2[['cid', 'alice_acute_class']], how = 'left', on = 'cid')
df3.drop_duplicates(inplace = True)
df3.head(10)

,cid,alice_category,chapter,group,category,subcategory,alice_acute_class
0,A01,NaN,Capítulo I - Algumas doenças infecciosas e par...,Doenças infecciosas intestinais,Febres tifóide e paratifóide,Febres tifóide e paratifóide,NaN
1,A02,Infecções gastrointestinais,Capítulo I - Algumas doenças infecciosas e par...,Doenças infecciosas intestinais,Outras infecções por Salmonella,Outras infecções por Salmonella,NaN
2,A022,NaN,Capítulo I - Algumas doenças infecciosas e par...,Doenças infecciosas intestinais,Outras infecções por Salmonella,Infecções localizadas por salmonela,NaN
3,A04,Infecções gastrointestinais,Capítulo I - Algumas doenças infecciosas e par...,Doenças infecciosas intestinais,Outras infecções intestinais bacterianas,Outras infecções intestinais bacterianas,NaN
4,A049,Infecções gastrointestinais,Capítulo I - Algumas doenças infecciosas e par...,Doenças infecciosas intestinais,Outras infecções intestinais bacterianas,Infecção intestinal bacteriana não especificada,agudo
5,A05,NaN,Capítulo I - Algumas doenças infecciosas e par...,Doenças infecciosas intestinais,"Outras intoxicações alimentares bacterianas, n...","Outras intoxicações alimentares bacterianas, n...",NaN
6,A059,Infecções gastrointestinais,Capítulo I - Algumas doenças infecciosas e par...,Doenças infecciosas intestinais,"Outras intoxicações alimentares bacterianas, n...",Intoxicação alimentar bacteriana não especificada,agudo
7,A06,Infecções gastrointestinais,Capítulo I - Algumas doenças infecciosas e par...,Doenças infecciosas intestinais,Amebíase,Amebíase,NaN
8,A069,NaN,Capítulo I - Algumas doenças infecciosas e par...,Doenças infecciosas intestinais,Amebíase,Amebíase não especificada,NaN
9,A071,Infecções gastrointestinais,Capítulo I - Algumas doenças infecciosas e par...,Doenças infecciosas intestinais,Outras doenças intestinais por protozoários,Giardíase [lamblíase],agudo


## 3. Export

In [31]:
di.redshift.insert_table(df3, table_name='alice_cid_categories', if_exists='replace')

Table alice_cid_categories inserted successfully, with 2907 rows.


In [32]:
df3['cid'].nunique()

2907

In [33]:
df3.count()

cid                  2907
alice_category       2494
chapter              2890
group                2890
category             2890
subcategory          2890
alice_acute_class    1909
dtype: int64